In [2]:
import cv2
import numpy as np
from my_function import imshow

## 图像形态学操作

图像形态学操作是基于图像形状的处理技术，主要用于二值图像的分析和处理。通过形态学操作实现对图像的噪声去除、对象分离、边缘检测等操作

常见的形态学操作如下：

|操作|函数|说明|应用场景|
| ----- | ----- | ----- | ----- |
|腐蚀（Erosion）|cv2.erode()|用结构元素扫描图像，如果结构元素覆盖的区域全是前景，则保留中心像素。|去除噪声、分离物体。|
|膨胀（Dilation）|cv2.dilate() |用结构元素扫描图像，如果结构元素覆盖的区域存在前景，则保留中心像素。| 连接断裂的物体、填充空洞。|
|开运算（Opening）|cv2.morphologyEx()|先腐蚀后膨胀。|去除小物体、平滑物体边界。|
|闭运算（Closing）|cv2.morphologyEx()|先膨胀后腐蚀。|填充小孔洞、连接邻近物体。|
|形态学梯度|cv2.morphologyEx()|膨胀图减去腐蚀图。|提取物体边缘。|
|顶帽运算|cv2.morphologyEx()|原图减去开运算结果。|提取比背景亮的细小物体。|
|黑帽运算|cv2.morphologyEx()|闭运算结果减去原图。|提取比背景暗的细小物体|



## 名词概念

### 前景

前景是指图像中我们感兴趣的主要对象或区域，通常是图像中需要被识别、分析或处理的部分。在图像处理中，前景通常与背景相对，背景是指图像中不感兴趣的部分

### 结构元素

结构元素是形态学操作中用于扫描和处理图像的一个小矩阵或模板。它定义了操作的形状和大小，决定了如何影响图像中的像素。常见的结构元素形状包括矩形、椭圆和十字形

### 中心像素

中心像素是指结构元素在图像上扫描时所覆盖区域的中心位置对应的像素。在形态学操作中，结构元素会在图像上移动，每次移动时都会考虑结构元素覆盖区域内的像素值，以决定中心像素的新值

## 腐蚀 erode()

- 腐蚀操作是一种缩小图像中前景对象的过程
- 腐蚀操作通过将结构元素与图像进行卷积，只有当结构元素完全覆盖图像中的前景像素时，中心像素才会被保留，否则会被腐蚀掉
- 腐蚀操作会使图像中的前景对象变小，边缘被腐蚀掉，常用于去除噪声或分离连接的对象

函数原型:

`cv2.erode(src, kernel, iterations=1)`

参数说明:

- src: 输入图像，通常是二值图像
- kernel: 结构元素，可以自定义或使用`cv2.getStructuringElement()`生成
- iterations: 腐蚀操作的次数，默认为1

函数原型:

`cv2.getStructuringElement(shape, ksize)`

参数说明:

- shape: 结构元素的形状，可以是`cv2.MORPH_RECT`（矩形）、`cv2.MORPH_ELLIPSE`（椭圆）或`cv2.MORPH_CROSS`（十字形）
- ksize: 结构元素的大小，以(宽度, 高度)的形式表示

In [3]:
# 对图像进行阈值处理
img=cv2.imread("./images/opencv_logo.jpg",cv2.IMREAD_GRAYSCALE)
_,binary=cv2.threshold(img,0,255,
        cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU) # 由于图片主体为黑色，故使用THRESH_BINARY_INV翻转

# 定义结构元素方法一
#kernel = np.ones((5,5), np.uint8)
# 定义结构元素方法二
kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

erode=cv2.erode(binary,kernel,iterations=3)
imshow(binary=binary,erode=erode)

## 膨胀 dilate()

- 膨胀操作与腐蚀相反，它是一种扩大图像中前景对象的过程
- 膨胀操作通过将结构元素与图像进行卷积，只要结构元素与图像中的前景像素有重叠，中心像素就会被保留
- 这会使图像中的前景对象变大，边缘被扩展，常用于填补前景对象中的空洞或连接断裂的对象

函数原型:

`cv2.dilate(src, kernel, iterations=1)`

参数说明:

- src: 输入图像，通常是二值图像
- kernel: 结构元素，可以自定义或使用`cv2.getStructuringElement()`
- iterations: 膨胀操作的次数，默认为1

In [4]:
img=cv2.imread("./images/opencv_logo.jpg",cv2.IMREAD_GRAYSCALE)
_,binary=cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

dilate=cv2.dilate(binary,kernel,iterations=3)
imshow(binary=binary,dilate=dilate)

## 开运算 MORPH_OPEN

- 开运算是先腐蚀后膨胀的组合操作
- 开运算可以去除图像中的小噪声，同时保留图像中的主要前景对象

函数原型:

`cv2.morphologyEx(src, op, kernel)`

参数说明:

- src: 输入图像，通常是二值图像
- op: 形态学操作类型，这里使用`cv2.MORPH_OPEN`
- kernel: 结构元素

In [5]:
_,binary=cv2.threshold(cv2.imread("./images/opening.png",cv2.IMREAD_GRAYSCALE),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))

open=cv2.morphologyEx(binary,cv2.MORPH_OPEN,kernel)
imshow(binary=binary,open=open)

## 闭运算 MORPH_CLOSE

- 闭运算是先膨胀后腐蚀的组合操作
- 闭运算可以填补前景对象中的小孔，同时保留图像中的主要前景对象

函数同上

In [6]:
_,binary=cv2.threshold(cv2.imread("./images/closing.png",cv2.IMREAD_GRAYSCALE),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))

close=cv2.morphologyEx(binary,cv2.MORPH_CLOSE,kernel)
imshow(binary=binary,close=close)

## 形态学梯度 MORPH_GRADIENT

- 形态学梯度是膨胀图像与腐蚀图像的差值
- 形态学梯度可以提取图像中前景对象的边缘，常用于边缘检测

函数同上

In [7]:
_,binary=cv2.threshold(cv2.imread("./images/opencv_logo.jpg",cv2.IMREAD_GRAYSCALE),0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))

gradient=cv2.morphologyEx(binary,cv2.MORPH_GRADIENT,kernel)
imshow(binary=binary,gradient=gradient)